In [ ]:
import ee
import folium
import os
import dotenv
from pprint import pprint
import geopandas as gpd
import geemap

In [ ]:
# Set up GEE credentials
dotenv.load_dotenv()

service_account = os.getenv('GCP_SERVICE_ACC')
private_key = os.getenv('GCP_SERVICE_JSON')

credentials = ee.ServiceAccountCredentials(service_account, private_key)

ee.Initialize(credentials)

In [ ]:
ee.Authenticate()
ee.Initialize()

In [ ]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee_image_object.getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [ ]:
# Get Snow Cover Data
sca: ee.ImageCollection = (ee.ImageCollection('MODIS/006/MOD10A1')
                           .filterDate('2023-01-01', '2023-01-4'))

# Get the number of images.
count = sca.size().getInfo()
print("Num Images:", count)

In [ ]:
# Create FeatureCollection of grid cells
aoi = (ee.Geometry.Polygon(
        [[[-120.88206223229643, 39.15849400460822],
          [-120.88206223229643, 39.149575297287804],
          [-120.87141922692534, 39.149575297287804],
          [-120.87141922692534, 39.15849400460822]]])) # Secret Town, CA

feature = ee.Feature(aoi, {'label': 'Secret Town'}) # Create feature from AOI
# proj = sca.first().projection() # Get projection from first image in collection
proj = feature.geometry().projection() # Get projection from feature

grid = feature.geometry().coveringGrid(proj, 500) # Create grid of 500x500m cells

In [ ]:
# Add grid cells to map

# map1 = folium.Map(location=[39.158763626894384, -120.88206515079846], zoom_start=14)
#
# map1.add_ee_layer(feature, {}, 'aoi')
# map1.add_ee_layer(grid, {}, 'grid')
# display(map1)

In [ ]:
# Def vis params for snow cover

# snowCoverVis = {
#   'min': 0,
#   'max': 100,
#   'palette': ['00FFFF', '0000FF'],
#   'opacity': 0.8
# }

In [ ]:
# map1.add_ee_layer(sca.select("NDSI_Snow_Cover").first(), snowCoverVis, 'sca')
# display(map1)

In [ ]:
# map2 = folium.Map(location=[39.158763626894384, -120.88206515079846], zoom_start=14)
#
# # Get NDSI_Snow_Cover for each grid cell
# def get_sca(image):
#     return image.reduceRegions(grid, ee.Reducer.mean(), 500)
#
# sca_grid = sca.map(get_sca).flatten()
# pprint(sca_grid.getInfo())
#
# # Add grid cells to map
# map2.add_ee_layer(sca_grid, snowCoverVis, 'sca')
# display(map2)

In [ ]:
url = grid.getDownloadURL(filetype="geojson", filename="grid")

print(url)

In [ ]:
def toCloudStorage(collection, bucket, folder=None, namePattern='{id}',
                   region=None, scale=30, dataType="float", datePattern=None,
                   verbose=False, extra=None, **kwargs):
    """ Upload all images from one collection to Google Cloud Storage. You can
    use the same arguments as the original function
    ee.batch.export.image.toCloudStorage

    :param collection: Collection to upload
    :type collection: ee.ImageCollection
    :param bucket: Google Cloud Storage bucket name
    :type folder: str
    :param folder: Google Cloud Storage prefix to export the images to
    :type folder: str
    :param namePattern: pattern for the name. See make_name function
    :type namePattern: str
    :param region: area to upload. Defualt to the footprint of the first
        image in the collection
    :type region: ee.Geometry.Rectangle or ee.Feature
    :param scale: scale of the image (side of one pixel). Defults to 30
        (Landsat resolution)
    :type scale: int
    :param dataType: as downloaded images **must** have the same data type
        in all bands, you have to set it here. Can be one of: "float",
        "double", "int", "Uint8", "Int8" or a casting function like
        *ee.Image.toFloat*
    :type dataType: str
    :param datePattern: pattern for date if specified in namePattern.
        Defaults to 'yyyyMMdd'
    :type datePattern: str
    """
    # empty tasks list
    tasklist = []
    # get region
    # region = tools.geometry.getRegion(region)
    # Make a list of images
    img_list = collection.toList(collection.size())

    n = 0
    while True:
        try:
            img = ee.Image(img_list.get(n))

            fc = ee.FeatureCollection(img.get('fc'))


            name = f"test_{n}"
            # name = name.getInfo()
            description = name
            # description = utils.matchDescription(name)

            # convert data type
            # img = utils.convertDataType(dataType)(img)

            if folder is not None:
                path = folder + "/" + name
            else:
                path = name

            url = fc.getDownloadURL(filetype="geojson", filename=name)
            print(url)

            # task = ee.batch.Export.table.toCloudStorage(collection=fc,
            #                                      description=description,
            #                                      bucket=bucket,
            #                                      path=path,
            #                                      **kwargs)
            # task.start()
            # tasklist.append(task)
            if verbose:
                print("adding task {} to list".format(name))
            n += 1

        except Exception as e:
            error = str(e).split(':')
            if error[0] == 'List.get':
                break
            else:
                raise e

    return tasklist

In [ ]:
def download_scas_regions(regions: ee.FeatureCollection, ee_ic: ee.ImageCollection):
    return ee_ic.map(lambda image: image.set('fc', image.reduceRegions(regions, ee.Reducer.mean(), 500)))

In [ ]:

# new_ic = download_scas_regions(grid, ee.ImageCollection(sca.first()))
new_ic = download_scas_regions(grid, sca)

first_fc = ee.FeatureCollection(new_ic.first().get('fc'))
first_fc.getDownloadUrl(filetype="geojson", filename="first_fc")

In [ ]:
out = toCloudStorage(new_ic, "none")

In [ ]:
ee.batch.Export.table.toAsset(first_fc, 'sca_fc', 'users/joshcchristensen7/sca_fc').start()

In [ ]:
list_fc = new_ic\
  .toList(5) \
  .map(lambda image: ee.FeatureCollection(ee.Image(image).get('fc')))

# list_fc = new_ic\
#   .toList(5) \
#   .map(lambda image: ee.batch.Export.table.toAsset(ee.FeatureCollection(ee.Image(image).get('fc')), 'sca_fc', 'users/joshcchristensen7/sca_fc'))

In [ ]:
fc = ee.FeatureCollection(list_fc)
fc.getInfo()

In [ ]:
import geetools

geetools.batch.Download.table.toLocal(fc, "test1234.geojson")

In [ ]:


tmp = sca.map(lambda image: image.reduceRegions(grid, ee.Reducer.mean(), 500))

tmp.map(lambda image: print(image.getDownloadURL(
    filetype='CSV',
    filename='sca',
    selectors=['NDSI_Snow_Cover']
)))

In [ ]:
grid_df = gpd.read_file("/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/grid.geojson")
grid_df.head()

In [ ]:
fc = geemap.geopandas_to_ee(grid_df)
pprint(fc.getInfo())

In [ ]:
# try downloading a big geojson
prediction_sites = gpd.read_file("/Users/jmac/Documents/Programming/REU/National-Snow-Model/Data_Processing_Assimilation/test.geojson")
prediction_sites.head()

In [ ]:
pred_sites_fc = geemap.geopandas_to_ee(prediction_sites[["geometry"]].head())
pprint(pred_sites_fc.getInfo())

In [ ]:
def wrapper (regions: ee.FeatureCollection):

    def wrapped (eeimage: ee.Image):
        return download_scas_regions(regions, eeimage)
    return wrapped

In [ ]:
downloader = wrapper(pred_sites_fc)

In [ ]:
downloads = sca.map(downloader).getInfo()